In [1]:
!pip install keras

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense, Conv1D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation, MaxPooling1D, Concatenate,Dropout, AveragePooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, UpSampling1D
from keras.regularizers import l2, l1, l1_l2

# The Speck cipher and data generation algorithms #
Taken from https://github.com/agohr/deep_speck/blob/master/speck.py

In [3]:
import numpy as np
from os import urandom

def WORD_SIZE():
    return(16);

def ALPHA():
    return(7);

def BETA():
    return(2);

MASK_VAL = 2 ** WORD_SIZE() - 1;

def shuffle_together(l):
    state = np.random.get_state();
    for x in l:
        np.random.set_state(state);
        np.random.shuffle(x);

def rol(x,k):
    return(((x << k) & MASK_VAL) | (x >> (WORD_SIZE() - k)));

def ror(x,k):
    return((x >> k) | ((x << (WORD_SIZE() - k)) & MASK_VAL));

def enc_one_round(p, k):
    c0, c1 = p[0], p[1];
    c0 = ror(c0, ALPHA());
    c0 = (c0 + c1) & MASK_VAL;
    c0 = c0 ^ k;
    c1 = rol(c1, BETA());
    c1 = c1 ^ c0;
    return(c0,c1);

def dec_one_round(c,k):
    c0, c1 = c[0], c[1];
    c1 = c1 ^ c0;
    c1 = ror(c1, BETA());
    c0 = c0 ^ k;
    c0 = (c0 - c1) & MASK_VAL;
    c0 = rol(c0, ALPHA());
    return(c0, c1);

def expand_key(k, t):
    ks = [0 for i in range(t)];
    ks[0] = k[len(k)-1];
    l = list(reversed(k[:len(k)-1]));
    for i in range(t-1):
        l[i%3], ks[i+1] = enc_one_round((l[i%3], ks[i]), i);
    return(ks);

def encrypt(p, ks):
    x, y = p[0], p[1];
    for k in ks:
        x,y = enc_one_round((x,y), k);
    return(x, y);

def decrypt(c, ks):
    x, y = c[0], c[1];
    for k in reversed(ks):
        x, y = dec_one_round((x,y), k);
    return(x,y);

def check_testvector():
  key = (0x1918,0x1110,0x0908,0x0100)
  pt = (0x6574, 0x694c)
  ks = expand_key(key, 22)
  ct = encrypt(pt, ks)
  if (ct == (0xa868, 0x42f2)):
    print("Testvector verified.")
    return(True);
  else:
    print("Testvector not verified.")
    return(False);

#convert_to_binary takes as input an array of ciphertext pairs
#where the first row of the array contains the lefthand side of the ciphertexts,
#the second row contains the righthand side of the ciphertexts,
#the third row contains the lefthand side of the second ciphertexts,
#and so on
#it returns an array of bit vectors containing the same data
def convert_to_binary(arr):
  X = np.zeros((4 * WORD_SIZE(),len(arr[0])),dtype=np.uint8);
  for i in range(4 * WORD_SIZE()):
    index = i // WORD_SIZE();
    offset = WORD_SIZE() - (i % WORD_SIZE()) - 1;
    X[i] = (arr[index] >> offset) & 1;
  X = X.transpose();
  return(X);

#takes a text file that contains encrypted block0, block1, true diff prob, real or random
#data samples are line separated, the above items whitespace-separated
#returns train data, ground truth, optimal ddt prediction
def readcsv(datei):
    data = np.genfromtxt(datei, delimiter=' ', converters={x: lambda s: int(s,16) for x in range(2)});
    X0 = [data[i][0] for i in range(len(data))];
    X1 = [data[i][1] for i in range(len(data))];
    Y = [data[i][3] for i in range(len(data))];
    Z = [data[i][2] for i in range(len(data))];
    ct0a = [X0[i] >> 16 for i in range(len(data))];
    ct1a = [X0[i] & MASK_VAL for i in range(len(data))];
    ct0b = [X1[i] >> 16 for i in range(len(data))];
    ct1b = [X1[i] & MASK_VAL for i in range(len(data))];
    ct0a = np.array(ct0a, dtype=np.uint16); ct1a = np.array(ct1a,dtype=np.uint16);
    ct0b = np.array(ct0b, dtype=np.uint16); ct1b = np.array(ct1b, dtype=np.uint16);
    
    #X = [[X0[i] >> 16, X0[i] & 0xffff, X1[i] >> 16, X1[i] & 0xffff] for i in range(len(data))];
    X = convert_to_binary([ct0a, ct1a, ct0b, ct1b]); 
    Y = np.array(Y, dtype=np.uint8); Z = np.array(Z);
    return(X,Y,Z);

#baseline training data generator
def make_train_data(n, nr, diff=(0x0040,0)):
  Y = np.frombuffer(urandom(n), dtype=np.uint8); Y = Y & 1;
  keys = np.frombuffer(urandom(8*n),dtype=np.uint16).reshape(4,-1);
  plain0l = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain0r = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain1l = plain0l ^ diff[0]; plain1r = plain0r ^ diff[1];
  num_rand_samples = np.sum(Y==0);
  plain1l[Y==0] = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  plain1r[Y==0] = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  ks = expand_key(keys, nr);
  ctdata0l, ctdata0r = encrypt((plain0l, plain0r), ks);
  ctdata1l, ctdata1r = encrypt((plain1l, plain1r), ks);
  X = convert_to_binary([ctdata0l, ctdata0r, ctdata1l, ctdata1r]);
  return(X,Y);

#real differences data generator
def real_differences_data(n, nr, diff=(0x0040,0)):
  #generate labels
  Y = np.frombuffer(urandom(n), dtype=np.uint8); Y = Y & 1;
  #generate keys
  keys = np.frombuffer(urandom(8*n),dtype=np.uint16).reshape(4,-1);
  #generate plaintexts
  plain0l = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain0r = np.frombuffer(urandom(2*n),dtype=np.uint16);
  #apply input difference
  plain1l = plain0l ^ diff[0]; plain1r = plain0r ^ diff[1];
  num_rand_samples = np.sum(Y==0);
  #expand keys and encrypt
  ks = expand_key(keys, nr);
  ctdata0l, ctdata0r = encrypt((plain0l, plain0r), ks);
  ctdata1l, ctdata1r = encrypt((plain1l, plain1r), ks);
  #generate blinding values
  k0 = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  k1 = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  #apply blinding to the samples labelled as random
  ctdata0l[Y==0] = ctdata0l[Y==0] ^ k0; ctdata0r[Y==0] = ctdata0r[Y==0] ^ k1;
  ctdata1l[Y==0] = ctdata1l[Y==0] ^ k0; ctdata1r[Y==0] = ctdata1r[Y==0] ^ k1;
  #convert to input data for neural networks
  X = convert_to_binary([ctdata0l, ctdata0r, ctdata1l, ctdata1r]);
  return(X,Y);


# Evaluate the results #
Taken from https://github.com/agohr/deep_speck/blob/master/eval.py and slightly adapted for evaluating the results of an autoencoder.

In [4]:
def evaluate(net,X,Y):
    
    Z = net.predict(X,batch_size=5000).flatten();
    Zbin = (Z > 0.5);

    #Compute the acc, tpr, tnr
    n = len(Z); 
    n0 = np.sum(Y==0); 
    n1 = np.sum(Y==1);
    
    acc = np.sum(Zbin == Y.flatten()) / n;
    tpr = np.sum(Zbin[Y.flatten()==1]) / n1;
    tnr = np.sum(Zbin[Y.flatten()==0] == 0) / n0;

    return(acc, tpr, tnr);

# Conduct multiple evaluations #


In [5]:
def multiple_evaluations(model, repetitions, num_rounds):
 
  #The accs, tprs, tnrs for all evaluation repetition
  accs = [];
  tprs = [];
  tnrs = [];
    
  #Evaluate multiple times and average the results 
  for i in range(0, repetitions):
    X_eval, Y_eval = make_train_data(10**6, num_rounds);

    (acc, tpr, tnr) = evaluate(model, X_eval, X_eval);
    accs.append(acc);
    tprs.append(tpr);
    tnrs.append(tnr);

  print("Acc: " + str(np.mean(accs)) + str(" +- ") + str(np.std(accs)) + str("\t") + 
        "Tpr:" + str(np.mean(tprs)) + str(" +- ") + str(np.std(tprs)) + str("\t") +
        "Tnr:" + str(np.mean(tnrs)) + str(" +- ") + str(np.std(tnrs)) + str("\t"));
        
  return(np.mean(accs), np.mean(tprs), np.mean(tnrs));

# Building the Autoencoder #
The implementation for all three versions is provided below (already unfolded)

In [6]:
#The encoder
def encode(input):

  x = Reshape((4, 16))(input);
  x = Permute((2,1))(x);

  #Block 1
  x = Conv1D(32, 3, padding='same')(x)
  x = BatchNormalization()(x);
  x = Activation('relu')(x);
  x = MaxPooling1D(2, padding='same')(x)

  #Block 2
  #x = Conv1D(32, 3, padding='same')(x)
  #x = BatchNormalization()(x);
  #x = Activation('relu')(x);
  #x = MaxPooling1D(2, padding='same')(x)
    
  #Block 3  
  #x = Conv1D(32, 3, padding='same')(x)
  #x = BatchNormalization()(x);
  #x = Activation('relu')(x);
  #x = MaxPooling1D(2, padding='same')(x)

  encoded = x;
    
  return encoded;


#The decoder
def decode(encoded):
  
  #Block 1 
  x = Conv1D(32, 3, padding='same')(encoded)
  x = BatchNormalization()(x);
  x = Activation('relu')(x);
  x = UpSampling1D(2)(x)

  #Block 2  
  #x = Conv1D(32, 3, padding='same')(x)
  #x = BatchNormalization()(x);
  #x = Activation('relu')(x);
  #x = UpSampling1D(2)(x)

  #Block 3
  #x = Conv1D(32, 3, padding='same')(x)
  #x = BatchNormalization()(x);
  #x = Activation('relu')(x);
  #x = UpSampling1D(2)(x)

 
  x = Conv1D(4, 3, activation='sigmoid', padding='same')(x)
  x = Permute((2,1))(x);
  decoded = Reshape((64,1))(x);

  return decoded;


#Using both the encoder and decoder to construct the autoencoder
def build_autoencoder():
    
  inp = Input(shape=(64,));

  encoded= encode(inp);
  decoded = decode(encoded);

  autoencoder = Model(inp, decoded);

  autoencoder.compile(
          optimizer='adam',
          loss='binary_crossentropy',
          metrics=['acc'])

  return autoencoder

For training the autoencoder \
Taken from https://github.com/agohr/deep_speck/blob/master/train_nets.py and slightly adapted.

In [7]:
#Batch size
bs=5000;

def cyclic_lr(num_epochs, high_lr, low_lr):
  res = lambda i: low_lr + ((num_epochs-1) - i % num_epochs)/(num_epochs-1) * (high_lr - low_lr);
  return(res);

def train_autoenc(model, num_epochs, num_rounds, X_train, Y_train, X_eval, Y_eval):
    
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, restore_best_weights= True);
    lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001));
    
    model.fit(X_train, Y_train, batch_size= bs, epochs=num_epochs,validation_data=(X_eval, Y_eval), callbacks=[lr, stop_early])
    
    return model;

# Repeat: Train the autoencoder and evaluate the results #

In [8]:
def repeat_experiment(repetitions, num_rounds):
 
  #Store the accs, tprs, tnrs for all experimenent repetitons
  accs = [];
  tprs = [];
  tnrs = [];
    
  #Repeat the experiment multiple times
  for i in range(0, repetitions):
    
    X_train, Y_train = make_train_data(10**7, num_rounds);
    X_eval, Y_eval = make_train_data(10**6, num_rounds);
    
    initial_model = build_autoencoder();
    #Set Y_train to X_train and Y_eval to X_eval
    trained_model= train_autoenc(initial_model, 30, num_rounds, X_train, X_train, X_eval, X_eval);

    (acc, tpr, tnr) = multiple_evaluations(trained_model,5,num_rounds);
    accs.append(acc);
    tprs.append(tpr);
    tnrs.append(tnr);
 
  print("\n") 
  print("Acc: " + str(np.mean(accs)) + str(" +- ") + str(np.std(accs)) + str("\t") + 
        "Tpr:" + str(np.mean(tprs)) + str(" +- ") + str(np.std(tprs)) + str("\t") +
        "Tnr:" + str(np.mean(tnrs)) + str(" +- ") + str(np.std(tnrs)) + str("\t"));
        
  return(np.mean(accs), np.mean(tprs), np.mean(tnrs));

# Run the experiment #

In [9]:
#num_rounds = 5, 6, 7, 8 with an autoencoder with 1/2/3 blocks (need to un/comment the blocks in the autoencoder implementation - un/comment both the block from the encoder and its conterpart from the decoder so that the ecoder and decoder will have the same number of blocks)
#num_repetitions = 5

repeat_experiment(repetitions = 2, num_rounds =5)

Epoch 1/30
2000/2000 [==============================] - 49s 8ms/step - loss: 0.1271 - acc: 0.9467 - val_loss: 0.0029 - val_acc: 0.9994
Epoch 2/30
2000/2000 [==============================] - 15s 8ms/step - loss: 0.0021 - acc: 0.9995 - val_loss: 0.0015 - val_acc: 0.9995
Epoch 3/30
2000/2000 [==============================] - 15s 8ms/step - loss: 0.0012 - acc: 0.9995 - val_loss: 0.0014 - val_acc: 0.9995
Epoch 4/30
2000/2000 [==============================] - 15s 8ms/step - loss: 9.5792e-04 - acc: 0.9996 - val_loss: 9.6071e-04 - val_acc: 0.9996
Epoch 5/30
2000/2000 [==============================] - 15s 8ms/step - loss: 8.6802e-04 - acc: 0.9996 - val_loss: 0.0010 - val_acc: 0.9995
Epoch 6/30
2000/2000 [==============================] - 16s 8ms/step - loss: 8.2468e-04 - acc: 0.9996 - val_loss: 9.1428e-04 - val_acc: 0.9996
Epoch 7/30
2000/2000 [==============================] - 15s 8ms/step - loss: 7.9684e-04 - acc: 0.9996 - val_loss: 8.9133e-04 - val_acc: 0.9996
Epoch 8/30
2000/2000 [=====

(0.9996200859375, 0.9996208617680671, 0.9996193086633007)

 ----------------------------------------------------------------------------------------------------------------------------

# Steps for training with preprocessed input  #

# 1. Train the autoencoder of choice # 

In [10]:
#Specify the number of rounds for which the autoencoder should be trained
num_rounds = 5;

X_train, Y_train = make_train_data(10**7, num_rounds);
X_eval, Y_eval = make_train_data(10**6, num_rounds);
    
initial_model = build_autoencoder();

#Set Y_train to X_train and Y_eval to X_eval
trained_model= train_autoenc(initial_model, 30, num_rounds, X_train, X_train, X_eval, X_eval);

Epoch 1/30
2000/2000 [==============================] - 17s 8ms/step - loss: 0.1275 - acc: 0.9473 - val_loss: 0.0032 - val_acc: 0.9993
Epoch 2/30
2000/2000 [==============================] - 15s 8ms/step - loss: 0.0022 - acc: 0.9995 - val_loss: 0.0019 - val_acc: 0.9994
Epoch 3/30
2000/2000 [==============================] - 15s 8ms/step - loss: 0.0012 - acc: 0.9995 - val_loss: 0.0014 - val_acc: 0.9995
Epoch 4/30
2000/2000 [==============================] - 15s 8ms/step - loss: 9.6302e-04 - acc: 0.9996 - val_loss: 0.0011 - val_acc: 0.9995
Epoch 5/30
2000/2000 [==============================] - 15s 8ms/step - loss: 8.6804e-04 - acc: 0.9996 - val_loss: 0.0012 - val_acc: 0.9995
Epoch 6/30
2000/2000 [==============================] - 16s 8ms/step - loss: 8.2239e-04 - acc: 0.9996 - val_loss: 9.2003e-04 - val_acc: 0.9996
Epoch 7/30
2000/2000 [==============================] - 15s 8ms/step - loss: 7.9309e-04 - acc: 0.9996 - val_loss: 7.9617e-04 - val_acc: 0.9996
Epoch 8/30
2000/2000 [=========

Let's look at a prediction

In [11]:
#Generate an instance and predict 
X, _ = make_train_data(1, num_rounds)
X_pred = trained_model.predict(X)

In [12]:
#Print the result
X_print= X[0];
X_pred_print = list(X_pred[0].flatten());

for i in range(64):
  print("At pozition: "+str(i) +"\t\t"+ "True value: " +str(X_print[i])+"\t\t" + "Predicted value: "+str(X_pred_print[i]));

At pozition: 0		True value: 1		Predicted value: 1.0
At pozition: 1		True value: 1		Predicted value: 1.0
At pozition: 2		True value: 0		Predicted value: 1.6348731e-16
At pozition: 3		True value: 1		Predicted value: 1.0
At pozition: 4		True value: 1		Predicted value: 0.9999999
At pozition: 5		True value: 0		Predicted value: 8.5918295e-12
At pozition: 6		True value: 1		Predicted value: 1.0
At pozition: 7		True value: 0		Predicted value: 9.8976045e-09
At pozition: 8		True value: 1		Predicted value: 1.0
At pozition: 9		True value: 0		Predicted value: 4.391558e-09
At pozition: 10		True value: 0		Predicted value: 2.2506596e-20
At pozition: 11		True value: 1		Predicted value: 1.0
At pozition: 12		True value: 1		Predicted value: 1.0
At pozition: 13		True value: 0		Predicted value: 7.566932e-09
At pozition: 14		True value: 0		Predicted value: 8.7065777e-13
At pozition: 15		True value: 0		Predicted value: 1.1346213e-11
At pozition: 16		True value: 0		Predicted value: 1.8506775e-21
At pozition: 17

# 2. Create an encoder model and set its weights to those of the encoder trained within the autoencoder from above #

In [13]:
#Create the encoder model
inpe = Input(shape=(64,));
oute = encode(inpe); 
encoder_model = Model(inpe,oute)

encoder_model.compile(
          optimizer='adam',
          loss='binary_crossentropy',
          metrics=['acc']);

#Set the weights to those of the pretrained encoder (from within the above-trained autoencoder)
for l1,l2 in zip(encoder_model.layers,trained_model.layers):
    l1.set_weights(l2.get_weights())
    print(l1.name)

input_4
reshape_6
permute_6
conv1d_9
batch_normalization_6
activation_6
max_pooling1d_3


# 3. Do not allow the weights of the encoder to change #

In [14]:
for layer in encoder_model.layers:
    layer.trainable = False

# 4. Add the preprocessing step to the (reduced/depth-1/10) distinguisher #
#### The network of the reduced distinguisher is given below.  
Taken from https://github.com/agohr/deep_speck/blob/master/train_nets.py and slightly adapted.

In [20]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense, Conv1D,Conv2D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation, MaxPooling1D, Concatenate,Dropout, AveragePooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, UpSampling1D
from keras.regularizers import l2, l1, l1_l2


def cyclic_lr(num_epochs, high_lr, low_lr):
  res = lambda i: low_lr + ((num_epochs-1) - i % num_epochs)/(num_epochs-1) * (high_lr - low_lr);
  return(res);

#Batch size
bs = 5000;


def make_resnet( num_blocks=2, num_filters=32, num_outputs=1, d1=64, d2=64, word_size=16, ks=3,depth=5, reg_param=0.0001, final_activation='sigmoid'):

  #Input and preprocessing layers
  inp = Input(shape=(64,));

  #Encode the input  
  x = encoder_model.call(inp); 

  
  #Please paste your network of choice within the dotted lines - here, the discovered reduced network is given
  #Also, do not forget to set the input of your network to x (the encoded inputs)  
  
  #---------------------------------------------------  
  #Block 1: 7 filters were pruned 
  conv1 = Conv1D(filters=25, kernel_size=1, padding='same')(x); #Also refered to as C1
  conv1 = BatchNormalization()(conv1);
  conv1 = Activation('relu')(conv1);

  #Block 2-i where:
  #From its first convolutional layer: 21 filters were pruned
  conv2 = Conv1D(filters=11, kernel_size=3, padding='same')(conv1); #Also refered to as C2
  conv2 = BatchNormalization()(conv2);
  conv2 = Activation('relu')(conv2);

  #From its second convolutional layer: 25 filters were pruned
  conv3 = Conv1D(filters=7, kernel_size=3, padding='same')(conv2); #Also refered to as C3
  conv3 = BatchNormalization()(conv3);
  conv3 = Activation('relu')(conv3);

  #The residual connection was removed 
    
  flat = Flatten()(conv3);

  #Block 3 where:
  #From its first dense layer: 46 neurons were pruned   
  dense1 = Dense(18)(flat); #Also refered to as D1
  dense1 = BatchNormalization()(dense1);
  dense1 = Activation('relu')(dense1);
    
  #From its second dense layer: 36 neurons were pruned  
  dense2 = Dense(28)(dense1); #Also refered to as D2
  dense2 = BatchNormalization()(dense2);
  dense2 = Activation('relu')(dense2);

  out = Dense(1, activation='sigmoid')(dense2);
  #---------------------------------------------------
    
    
  model = Model(inputs=inp, outputs=out);

  return model;



def model_builder(depth):

  model = make_resnet(depth=depth);

  model.compile(
          optimizer='adam',
          loss='binary_crossentropy',
          metrics=['acc']);
    
  return model;
  



def train_speck_distinguisher(model, num_epochs, num_rounds, X_train, Y_train, X_eval, Y_eval):
    
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, restore_best_weights= True);
    lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001));
    
    model.fit(X_train, Y_train, batch_size= bs, epochs=num_epochs, validation_data=(X_eval, Y_eval), callbacks=[lr, stop_early])
    
    return model;


# 5. Run the experiment #
You can take the network (without the preprocessing of the inputs) directly from https://github.com/agohr/deep_speck/blob/master/train_nets.py 
\
Change the depth to 10 or 1 to conduct the other experiments, or take the detpth-1/10 network from the end of this notebook.\

For the Demo, first the reduced network is used as a classifier, and then the depth-10 network.

In [18]:
#Here, the reduced network is used as a classifier. The results are given below. 
initial_model_reduced_network = model_builder(depth=1); 
trained_model_reduced_network= train_speck_distinguisher(initial_model_reduced_network, 30, num_rounds, X_train, Y_train, X_eval, Y_eval);

Epoch 1/30
2000/2000 [==============================] - 49s 8ms/step - loss: 0.4815 - acc: 0.7639 - val_loss: 0.3548 - val_acc: 0.8531
Epoch 2/30
2000/2000 [==============================] - 15s 8ms/step - loss: 0.3366 - acc: 0.8612 - val_loss: 0.3274 - val_acc: 0.8653
Epoch 3/30
2000/2000 [==============================] - 15s 8ms/step - loss: 0.3193 - acc: 0.8694 - val_loss: 0.3172 - val_acc: 0.8713
Epoch 4/30
2000/2000 [==============================] - 16s 8ms/step - loss: 0.3118 - acc: 0.8732 - val_loss: 0.3117 - val_acc: 0.8731
Epoch 5/30
2000/2000 [==============================] - 16s 8ms/step - loss: 0.3074 - acc: 0.8751 - val_loss: 0.3074 - val_acc: 0.8752
Epoch 6/30
2000/2000 [==============================] - 16s 8ms/step - loss: 0.3037 - acc: 0.8768 - val_loss: 0.3037 - val_acc: 0.8768
Epoch 7/30
2000/2000 [==============================] - 16s 8ms/step - loss: 0.3010 - acc: 0.8780 - val_loss: 0.3010 - val_acc: 0.8779
Epoch 8/30
2000/2000 [==============================] -

In [19]:
#Evaluate the results
multiple_evaluations_model_with_preprocessing(model = trained_model_reduced_network, repetitions = 2, num_rounds = num_rounds)

Acc: 0.887205 +- 8.399999999997299e-05	Tpr:0.8361174308641061 +- 5.45802513906013e-05	Tnr:0.9384257290957074 +- 2.000469678131722e-05	


(0.887205, 0.8361174308641061, 0.9384257290957074)

In [21]:
#Here, the depth-10 network is used as a classifier. The results are given below. 
initial_model_depth10 = model_builder(depth=10); 
trained_model_depth10 = train_speck_distinguisher(initial_model_depth10, 30, num_rounds, X_train, Y_train, X_eval, Y_eval);

Epoch 1/30
2000/2000 [==============================] - 73s 30ms/step - loss: 0.4710 - acc: 0.8031 - val_loss: 0.3084 - val_acc: 0.8844
Epoch 2/30
2000/2000 [==============================] - 60s 30ms/step - loss: 0.2860 - acc: 0.8935 - val_loss: 0.2905 - val_acc: 0.8915
Epoch 3/30
2000/2000 [==============================] - 60s 30ms/step - loss: 0.2670 - acc: 0.9013 - val_loss: 0.2628 - val_acc: 0.9032
Epoch 4/30
2000/2000 [==============================] - 60s 30ms/step - loss: 0.2552 - acc: 0.9051 - val_loss: 0.3152 - val_acc: 0.8787
Epoch 5/30
2000/2000 [==============================] - 60s 30ms/step - loss: 0.2519 - acc: 0.9054 - val_loss: 0.2507 - val_acc: 0.9060
Epoch 6/30
2000/2000 [==============================] - 60s 30ms/step - loss: 0.2394 - acc: 0.9097 - val_loss: 0.2826 - val_acc: 0.8894
Epoch 7/30
2000/2000 [==============================] - 60s 30ms/step - loss: 0.2360 - acc: 0.9103 - val_loss: 0.2363 - val_acc: 0.9099
Epoch 8/30
2000/2000 [==========================

In [22]:
#Evaluate the results
multiple_evaluations_model_with_preprocessing(model = trained_model_depth10, repetitions = 2, num_rounds = num_rounds)

Acc: 0.9108855 +- 3.3500000000019625e-05	Tpr:0.8742460440545509 +- 0.00013561803774508885	Tnr:0.9476719777844262 +- 0.00015304689820411221	


(0.9108855, 0.8742460440545509, 0.9476719777844262)

The evaluate function for evaluating the model with the encoder as a preprocessor.\
Taken from https://github.com/agohr/deep_speck/blob/master/eval.py and slightly adapted.

Evaluate the results

In [16]:
def evaluate_model_with_preprocessing(net,X,Y):
    
    Z = net.predict(X,batch_size=10000).flatten();
    Zbin = (Z > 0.5);
    
    #Compute the acc, tpr, tnr
    n = len(Z); 
    n0 = np.sum(Y==0); 
    n1 = np.sum(Y==1);
    
    acc = np.sum(Zbin == Y) / n;
    tpr = np.sum(Zbin[Y==1]) / n1;
    tnr = np.sum(Zbin[Y==0] == 0) / n0;
    
    return(acc, tpr, tnr); 

Conduct multiple evaluations

In [17]:
def multiple_evaluations_model_with_preprocessing(model, repetitions, num_rounds):
 
  #The accs, tprs, tnrs for all evaluation repetition
  accs = [];
  tprs = [];
  tnrs = [];
    
  #Evaluate multiple times and average the results 
  for i in range(0, repetitions):
    X_eval, Y_eval = make_train_data(10**6, num_rounds);

    (acc, tpr, tnr) = evaluate_model_with_preprocessing(model, X_eval, Y_eval);
    accs.append(acc);
    tprs.append(tpr);
    tnrs.append(tnr);

  print("Acc: " + str(np.mean(accs)) + str(" +- ") + str(np.std(accs)) + str("\t") + 
        "Tpr:" + str(np.mean(tprs)) + str(" +- ") + str(np.std(tprs)) + str("\t") +
        "Tnr:" + str(np.mean(tnrs)) + str(" +- ") + str(np.std(tnrs)) + str("\t"));
        
  return(np.mean(accs), np.mean(tprs), np.mean(tnrs));

Gohr's network for conducting the above experiment with the depth-1/10 distinguisher \
Taken from https://github.com/agohr/deep_speck/blob/master/train_nets.py

In [ ]:
  #Block 1
  conv0 = Conv1D(num_filters, kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(x);
  conv0 = BatchNormalization()(conv0);
  conv0 = Activation('relu')(conv0);

  #Blocks 2-i - residual blocks
  shortcut = conv0;
  for i in range(depth):
    conv1 = Conv1D(num_filters, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut);
    conv1 = BatchNormalization()(conv1);
    conv1 = Activation('relu')(conv1);
    
    conv2 = Conv1D(num_filters, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(conv1);
    conv2 = BatchNormalization()(conv2);
    conv2 = Activation('relu')(conv2);
    shortcut = Add()([shortcut, conv2]);
    
  #Block 3
  flat1 = Flatten()(shortcut);
    
  dense1 = Dense(d1,kernel_regularizer=l2(reg_param))(flat1);
  dense1 = BatchNormalization()(dense1);
  dense1 = Activation('relu')(dense1);

  dense2 = Dense(d2, kernel_regularizer=l2(reg_param))(dense1);
  dense2 = BatchNormalization()(dense2);
  dense2 = Activation('relu')(dense2);
    
  out = Dense(num_outputs, activation=final_activation, kernel_regularizer=l2(reg_param))(dense2);